In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, f_oneway
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*80)
print("ALPHACARE INSURANCE SOLUTIONS - COMPLETE ANALYSIS PIPELINE")
print("="*80)

ALPHACARE INSURANCE SOLUTIONS - COMPLETE ANALYSIS PIPELINE


In [3]:
# Peek at first few lines
with open('./Data/MachineLearningRating_v3.txt', 'r') as f:
    for i, line in enumerate(f):
        if i < 5:  # Show first 5 lines
            print(repr(line))  # repr() shows hidden characters like tabs
        else:
            break

'UnderwrittenCoverID|PolicyID|TransactionMonth|IsVATRegistered|Citizenship|LegalType|Title|Language|Bank|AccountType|MaritalStatus|Gender|Country|Province|PostalCode|MainCrestaZone|SubCrestaZone|ItemType|mmcode|VehicleType|RegistrationYear|make|Model|Cylinders|cubiccapacity|kilowatts|bodytype|NumberOfDoors|VehicleIntroDate|CustomValueEstimate|AlarmImmobiliser|TrackingDevice|CapitalOutstanding|NewVehicle|WrittenOff|Rebuilt|Converted|CrossBorder|NumberOfVehiclesInFleet|SumInsured|TermFrequency|CalculatedPremiumPerTerm|ExcessSelected|CoverCategory|CoverType|CoverGroup|Section|Product|StatutoryClass|StatutoryRiskType|TotalPremium|TotalClaims\n'
'145249|12827|2015-03-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current account|Not specified|Not specified|South Africa|Gauteng|1459|Rand East|Rand East|Mobility - Motor|44069150|Passenger Vehicle|2004|MERCEDES-BENZ|E 240|6|2597|130|S/D|4|6/2002|119300|Yes|No|119300|More than 6 months||||||0.01|Monthly|25|Mobility - Winds

In [4]:
# ============================================================================
# TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING
# ============================================================================

print("\n" + "="*80)
print("TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING")
print("="*80)

# Load the data
print("\n[1.1] Loading data...")
df = pd.read_csv('./Data/MachineLearningRating_v3.txt', sep='|',encoding='utf-8-sig') 



print("Data loaded successfully!")
print(f"Dataset shape: {df.shape}")

print("\n[1.2] Data Understanding - Basic Information")
print("-" * 80)
print(df.info())
print("\nFirst few rows:")
print(df.head())


TASK 1: EXPLORATORY DATA ANALYSIS & STATISTICAL THINKING

[1.1] Loading data...
Data loaded successfully!
Dataset shape: (1000098, 52)

[1.2] Data Understanding - Basic Information
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   objec

In [5]:
print("\n[1.3] Data Quality Assessment")
print("-" * 80)
# Function to assess data quality
def assess_data_quality(df):
    """Comprehensive data quality assessment"""
    quality_report = pd.DataFrame({
        'Column': df.columns,
        'Data_Type': df.dtypes,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
        'Unique_Values': df.nunique(),
        'Sample_Values': [df[col].dropna().unique()[:3].tolist() if len(df[col].dropna().unique()) > 0 else [] for col in df.columns]
    })
    
    print("Data Quality Report:")
    print(quality_report[quality_report.Missing_Count > 0].sort_values('Missing_Percentage', ascending=False))
    
    return quality_report
quality_report = assess_data_quality(df)


[1.3] Data Quality Assessment
--------------------------------------------------------------------------------
Data Quality Report:
                                          Column Data_Type  Missing_Count  \
NumberOfVehiclesInFleet  NumberOfVehiclesInFleet   float64        1000098   
CrossBorder                          CrossBorder    object         999400   
CustomValueEstimate          CustomValueEstimate   float64         779642   
WrittenOff                            WrittenOff    object         641901   
Converted                              Converted    object         641901   
Rebuilt                                  Rebuilt    object         641901   
NewVehicle                            NewVehicle    object         153295   
Bank                                        Bank    object         145961   
AccountType                          AccountType    object          40232   
Gender                                    Gender    object           9536   
MaritalStatus       

In [6]:
print("DataFrame shape:", df.shape)
print("Columns:", list(df.columns))
print("\nTotal missing values per column:")
print(df.isnull().sum())
print("\nAny missing at all?", df.isnull().values.any())

DataFrame shape: (1000098, 52)
Columns: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']

Total missing values per column:
UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered 

In [7]:
print("\n[1.4] Descriptive Statistics")
print("-" * 80)

def calculate_descriptive_stats(df):
    """Calculate comprehensive descriptive statistics"""
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    stats_summary = df[numerical_cols].describe().T
    stats_summary['variance'] = df[numerical_cols].var()
    stats_summary['skewness'] = df[numerical_cols].skew()
    stats_summary['kurtosis'] = df[numerical_cols].kurtosis()
    
    print("Descriptive Statistics for Numerical Variables:")
    print(stats_summary)
    
    return stats_summary
stats_summary = calculate_descriptive_stats(df)


[1.4] Descriptive Statistics
--------------------------------------------------------------------------------
Descriptive Statistics for Numerical Variables:
                              count          mean           std           min  \
UnderwrittenCoverID       1000098.0  1.048175e+05  6.329371e+04  1.000000e+00   
PolicyID                  1000098.0  7.956682e+03  5.290039e+03  1.400000e+01   
PostalCode                1000098.0  3.020601e+03  2.649854e+03  1.000000e+00   
mmcode                     999546.0  5.487770e+07  1.360381e+07  4.041200e+06   
RegistrationYear          1000098.0  2.010225e+03  3.261391e+00  1.987000e+03   
Cylinders                  999546.0  4.046642e+00  2.940201e-01  0.000000e+00   
cubiccapacity              999546.0  2.466743e+03  4.428006e+02  0.000000e+00   
kilowatts                  999546.0  9.720792e+01  1.939326e+01  0.000000e+00   
NumberOfDoors              999546.0  4.019250e+00  4.683144e-01  0.000000e+00   
CustomValueEstimate        2204

In [8]:
print("\n[1.5] Key Business Metrics Calculation")
print("-" * 80)

def calculate_business_metrics(df):
    """Calculate key insurance business metrics"""
    
    # Overall metrics
    overall_metrics = {
        'Total_Premium': df['TotalPremium'].sum(),
        'Total_Claims': df['TotalClaims'].sum(),
        'Overall_Loss_Ratio': df['TotalClaims'].sum() / df['TotalPremium'].sum(),
        'Number_of_Policies': df['PolicyID'].nunique(),
        'Claim_Frequency': (df['TotalClaims'] > 0).mean(),
        'Average_Premium': df['TotalPremium'].mean(),
        'Average_Claim': df[df['TotalClaims'] > 0]['TotalClaims'].mean(),
        'Total_Margin': (df['TotalPremium'] - df['TotalClaims']).sum()
    }
    
    print("Overall Business Metrics:")
    for metric, value in overall_metrics.items():
        if 'Ratio' in metric or 'Frequency' in metric:
            print(f"{metric}: {value:.2%}")
        else:
            print(f"{metric}: {value:,.2f}")
    
    # Loss ratio by Province
    print("\n\nLoss Ratio by Province:")
    loss_by_province = df.groupby('Province').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_province['LossRatio'] = loss_by_province['TotalClaims'] / loss_by_province['TotalPremium']
    loss_by_province = loss_by_province.sort_values('LossRatio', ascending=False)
    print(loss_by_province)
    
    # Loss ratio by VehicleType
    print("\n\nLoss Ratio by Vehicle Type:")
    loss_by_vehicle = df.groupby('VehicleType').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_vehicle['LossRatio'] = loss_by_vehicle['TotalClaims'] / loss_by_vehicle['TotalPremium']
    loss_by_vehicle = loss_by_vehicle.sort_values('LossRatio', ascending=False)
    print(loss_by_vehicle)
    
    # Loss ratio by Gender
    print("\n\nLoss Ratio by Gender:")
    loss_by_gender = df.groupby('Gender').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    })
    loss_by_gender['LossRatio'] = loss_by_gender['TotalClaims'] / loss_by_gender['TotalPremium']
    print(loss_by_gender)
    
    return overall_metrics, loss_by_province, loss_by_vehicle, loss_by_gender

overall_metrics, loss_by_province, loss_by_vehicle, loss_by_gender = calculate_business_metrics(df)



[1.5] Key Business Metrics Calculation
--------------------------------------------------------------------------------
Overall Business Metrics:
Total_Premium: 61,911,562.70
Total_Claims: 64,867,546.17
Overall_Loss_Ratio: 104.77%
Number_of_Policies: 7,000.00
Claim_Frequency: 0.28%
Average_Premium: 61.91
Average_Claim: 23,273.39
Total_Margin: -2,955,983.47


Loss Ratio by Province:
                TotalClaims  TotalPremium  PolicyID  LossRatio
Province                                                      
Gauteng        2.939415e+07  2.405377e+07    393865   1.222018
KwaZulu-Natal  1.430138e+07  1.320908e+07    169781   1.082693
Western Cape   1.038977e+07  9.806559e+06    170796   1.059472
North West     5.920250e+06  7.490508e+06    143287   0.790367
Mpumalanga     2.044675e+06  2.836292e+06     52718   0.720897
Free State     3.549223e+05  5.213632e+05      8099   0.680758
Limpopo        1.016477e+06  1.537324e+06     24836   0.661199
Eastern Cape   1.356427e+06  2.140104e+06     3

In [9]:
print("\n[1.6] Univariate Analysis - Distributions")
print("-" * 80)

def univariate_analysis(df):
    """Perform univariate analysis with visualizations"""
    
    numerical_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                      'CalculatedPremiumPerTerm', 'CustomValueEstimate']
    
    # Histograms
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, col in enumerate(numerical_cols):
        if col in df.columns:
            axes[idx].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('univariate_distributions.png', dpi=300, bbox_inches='tight')
    print("Saved: univariate_distributions.png")
    plt.close()
    
    # Box plots for outlier detection
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, col in enumerate(['TotalPremium', 'TotalClaims', 'CustomValueEstimate']):
        if col in df.columns:
            axes[idx].boxplot(df[col].dropna())
            axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
            axes[idx].set_ylabel(col)
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('boxplots_outliers.png', dpi=300, bbox_inches='tight')
    print("Saved: boxplots_outliers.png")
    plt.close()
    
    # Categorical variables
    categorical_cols = ['Province', 'VehicleType', 'Gender', 'CoverType']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, col in enumerate(categorical_cols):
        if col in df.columns:
            value_counts = df[col].value_counts().head(10)
            axes[idx].bar(range(len(value_counts)), value_counts.values)
            axes[idx].set_xticks(range(len(value_counts)))
            axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
            axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[idx].set_ylabel('Count')
            axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('categorical_distributions.png', dpi=300, bbox_inches='tight')
    print("Saved: categorical_distributions.png")
    plt.close()

univariate_analysis(df)


[1.6] Univariate Analysis - Distributions
--------------------------------------------------------------------------------
Saved: univariate_distributions.png
Saved: boxplots_outliers.png
Saved: categorical_distributions.png


In [10]:
print("\n[1.7] Bivariate and Multivariate Analysis")
print("-" * 80)

def bivariate_analysis(df):
    """Perform bivariate and multivariate analysis"""
    
    # Correlation matrix
    numerical_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                      'CalculatedPremiumPerTerm', 'CustomValueEstimate']
    
    if all(col in df.columns for col in numerical_cols):
        corr_matrix = df[numerical_cols].corr()
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                    square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Correlation Matrix - Key Financial Variables', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
        print("Saved: correlation_matrix.png")
        plt.close()
    
    # Premium vs Claims by PostalCode
    postal_analysis = df.groupby('PostalCode').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum'
    }).reset_index()
    
    plt.figure(figsize=(14, 8))
    plt.scatter(postal_analysis['TotalPremium'], 
               postal_analysis['TotalClaims'], 
               alpha=0.6, s=100, edgecolors='black', linewidth=0.5)
    plt.xlabel('Total Premium', fontsize=12)
    plt.ylabel('Total Claims', fontsize=12)
    plt.title('Premium vs Claims by PostalCode', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Add diagonal line
    max_val = max(postal_analysis['TotalPremium'].max(), postal_analysis['TotalClaims'].max())
    plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Break-even line')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('premium_vs_claims_postal.png', dpi=300, bbox_inches='tight')
    print("Saved: premium_vs_claims_postal.png")
    plt.close()
    
    # Temporal trends
    df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
    df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)
    
    monthly_trends = df.groupby('TransactionMonth').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'HasClaim': 'mean'
    }).reset_index()
    
    fig, ax1 = plt.subplots(figsize=(16, 8))
    ax2 = ax1.twinx()
    
    ax1.plot(monthly_trends['TransactionMonth'], 
             monthly_trends['TotalClaims'], 
             'b-', linewidth=2, label='Total Claims', marker='o')
    ax2.plot(monthly_trends['TransactionMonth'], 
             monthly_trends['HasClaim'], 
             'r-', linewidth=2, label='Claim Frequency', marker='s')
    
    ax1.set_xlabel('Month', fontsize=12)
    ax1.set_ylabel('Total Claims', color='b', fontsize=12)
    ax2.set_ylabel('Claim Frequency', color='r', fontsize=12)
    ax1.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    
    plt.title('Temporal Trends in Claims and Frequency', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.tight_layout()
    plt.savefig('temporal_trends.png', dpi=300, bbox_inches='tight')
    print("Saved: temporal_trends.png")
    plt.close()
bivariate_analysis(df)


[1.7] Bivariate and Multivariate Analysis
--------------------------------------------------------------------------------
Saved: correlation_matrix.png
Saved: premium_vs_claims_postal.png
Saved: temporal_trends.png


In [11]:
print("\n[1.8] Outlier Detection")
print("-" * 80)

def detect_outliers(df):
    """Detect outliers using IQR method"""
    
    numerical_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate']
    
    outlier_summary = []
    
    for col in numerical_cols:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            
            outlier_summary.append({
                'Column': col,
                'Lower_Bound': lower_bound,
                'Upper_Bound': upper_bound,
                'Outlier_Count': len(outliers),
                'Outlier_Percentage': len(outliers) / len(df) * 100
            })
    
    outlier_df = pd.DataFrame(outlier_summary)
    print("Outlier Detection Summary (IQR Method):")
    print(outlier_df)
    
    return outlier_df
outlier_df = detect_outliers(df)


[1.8] Outlier Detection
--------------------------------------------------------------------------------
Outlier Detection Summary (IQR Method):
                Column   Lower_Bound    Upper_Bound  Outlier_Count  \
0         TotalPremium    -32.894737      54.824561         209042   
1          TotalClaims      0.000000       0.000000           2793   
2  CustomValueEstimate -82500.000000  497500.000000           1785   

   Outlier_Percentage  
0           20.902152  
1            0.279273  
2            0.178483  


In [12]:
def find_column(df, possible_names):
    """
    Find column in DataFrame by checking multiple possible names (case-insensitive)
    """
    for name in possible_names:
        # Check exact match
        if name in df.columns:
            return name
        # Check case-insensitive match
        for col in df.columns:
            if col.lower() == name.lower():
                return col
            # Check with spaces/underscores removed
            if col.lower().replace(' ', '').replace('_', '') == name.lower().replace(' ', '').replace('_', ''):
                return col
    return None

In [13]:
print("\n[1.9] Creative Visualizations")
print("-" * 80)
def create_creative_visualizations(df):
    # Reuse column mapping logic
    col_map = {
        'Province': find_column(df, ['Province']),
        'VehicleType': find_column(df, ['VehicleType']),
        'TotalClaims': find_column(df, ['TotalClaims']),
        'TotalPremium': find_column(df, ['TotalPremium']),
        'PolicyID': find_column(df, ['PolicyID', 'UnderwrittenCoverID']),
        'Make': find_column(df, ['Make']),
        'Model': find_column(df, ['Model']),
        'PostalCode': find_column(df, ['PostalCode']),
        'Gender': find_column(df, ['Gender']),
    }

    print("Column mapping:")
    for k, v in col_map.items():
        print(f"  {'✓' if v else '✗'} {k:15s} -> {v}")

    # Visualization 1
    if col_map['Province'] and col_map['VehicleType'] and col_map['TotalClaims']:
        pivot_data = df.pivot_table(
            values=col_map['TotalClaims'],
            index=col_map['Province'],
            columns=col_map['VehicleType'],
            aggfunc='mean'
        )
        plt.figure(figsize=(14, 8))
        sns.heatmap(pivot_data, annot=True, fmt='.0f', cmap='RdYlGn_r', 
                    linewidths=0.5, cbar_kws={'label': 'Average Claim Amount'})
        plt.title('Risk Profile: Average Claim Amount by Province and Vehicle Type', 
                  fontsize=14, fontweight='bold')
        plt.xlabel('Vehicle Type', fontsize=12)
        plt.ylabel('Province', fontsize=12)
        plt.tight_layout()
        plt.savefig('creative_viz_1_risk_heatmap.png', dpi=300, bbox_inches='tight')
        print("Saved: creative_viz_1_risk_heatmap.png")
        plt.close()
    else:
        print("Skipping Visualization 1: missing required columns")

    # Visualization 2
    if all(col_map[k] for k in ['TotalPremium','TotalClaims','Province','PolicyID']):
        df_viz2 = df.copy()
        df_viz2['Margin'] = df_viz2[col_map['TotalPremium']] - df_viz2[col_map['TotalClaims']]
        province_profit = df_viz2.groupby(col_map['Province']).agg({
            'Margin': 'sum',
            col_map['TotalPremium']: 'sum',
            col_map['PolicyID']: 'count'
        }).reset_index()
        province_profit.columns = ['Province', 'Margin', 'TotalPremium', 'PolicyCount']
        province_profit['AvgMarginPerPolicy'] = province_profit['Margin'] / province_profit['PolicyCount']
        province_profit = province_profit.sort_values('AvgMarginPerPolicy', ascending=False)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
        colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in province_profit['AvgMarginPerPolicy']]
        ax1.barh(province_profit['Province'], province_profit['AvgMarginPerPolicy'], color=colors, alpha=0.7)
        ax1.set_xlabel('Average Margin per Policy', fontsize=12)
        ax1.set_title('Profitability by Province', fontsize=13, fontweight='bold')
        ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)
        ax1.grid(True, alpha=0.3)
        ax2.scatter(province_profit['TotalPremium'], province_profit['Margin'], 
                   s=province_profit['PolicyCount']*3, alpha=0.6, c=colors, edgecolors='black')
        ax2.set_xlabel('Total Premium', fontsize=12)
        ax2.set_ylabel('Total Margin', fontsize=12)
        ax2.set_title('Premium vs Margin (size = policy count)', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('creative_viz_2_profitability.png', dpi=300, bbox_inches='tight')
        print("Saved: creative_viz_2_profitability.png")
        plt.close()
    else:
        print("Skipping Visualization 2: missing required columns")

    # Visualization 3
    if col_map['Make'] and col_map['TotalClaims'] and col_map['TotalPremium'] and col_map['PolicyID']:
        groupby_cols = [col_map['Make']]
        if col_map['Model']:
            groupby_cols.append(col_map['Model'])
        vehicle_analysis = df.groupby(groupby_cols).agg({
            col_map['TotalClaims']: 'sum',
            col_map['TotalPremium']: 'sum',
            col_map['PolicyID']: 'count'
        }).reset_index()
        suffix = ['TotalClaims','TotalPremium','PolicyCount']
        vehicle_analysis.columns = groupby_cols + suffix
        vehicle_analysis = vehicle_analysis[vehicle_analysis['PolicyCount'] >= 5]  # relaxed threshold
        if len(vehicle_analysis) > 0:
            vehicle_analysis['LossRatio'] = vehicle_analysis['TotalClaims'] / vehicle_analysis['TotalPremium']
            vehicle_analysis['AvgPremium'] = vehicle_analysis['TotalPremium'] / vehicle_analysis['PolicyCount']
            vehicle_analysis = vehicle_analysis[(vehicle_analysis['LossRatio'] >= 0) & (vehicle_analysis['LossRatio'] < 5)]
            if len(vehicle_analysis) > 0:
                plt.figure(figsize=(16, 10))
                scatter = plt.scatter(vehicle_analysis['AvgPremium'], 
                                     vehicle_analysis['LossRatio'],
                                     s=vehicle_analysis['PolicyCount']*5,
                                     c=vehicle_analysis['LossRatio'],
                                     cmap='RdYlGn_r',
                                     alpha=0.6,
                                     edgecolors='black',
                                     linewidth=0.5)
                plt.colorbar(scatter, label='Loss Ratio')
                plt.xlabel('Average Premium per Policy', fontsize=12)
                plt.ylabel('Loss Ratio', fontsize=12)
                title = f'Customer Segmentation: Risk vs Value ({" and ".join(groupby_cols)})'
                plt.title(title, fontsize=14, fontweight='bold')
                plt.axhline(y=1.0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Break-even')
                plt.axvline(x=vehicle_analysis['AvgPremium'].median(), color='blue', 
                           linestyle='--', linewidth=1, alpha=0.5, label='Median Premium')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig('creative_viz_3_segmentation.png', dpi=300, bbox_inches='tight')
                print("Saved: creative_viz_3_segmentation.png")
                plt.close()
            else:
                print("Skipping Visualization 3: insufficient valid data after filtering")
        else:
            print("Skipping Visualization 3: too few vehicle groups")
    else:
        print("Skipping Visualization 3: missing Make/TotalClaims/TotalPremium/PolicyID")

create_creative_visualizations(df)


[1.9] Creative Visualizations
--------------------------------------------------------------------------------
Column mapping:
  ✓ Province        -> Province
  ✓ VehicleType     -> VehicleType
  ✓ TotalClaims     -> TotalClaims
  ✓ TotalPremium    -> TotalPremium
  ✓ PolicyID        -> PolicyID
  ✓ Make            -> make
  ✓ Model           -> Model
  ✓ PostalCode      -> PostalCode
  ✓ Gender          -> Gender
Saved: creative_viz_1_risk_heatmap.png
Saved: creative_viz_2_profitability.png
Saved: creative_viz_3_segmentation.png


In [14]:
create_creative_visualizations(df)


Column mapping:
  ✓ Province        -> Province
  ✓ VehicleType     -> VehicleType
  ✓ TotalClaims     -> TotalClaims
  ✓ TotalPremium    -> TotalPremium
  ✓ PolicyID        -> PolicyID
  ✓ Make            -> make
  ✓ Model           -> Model
  ✓ PostalCode      -> PostalCode
  ✓ Gender          -> Gender
Saved: creative_viz_1_risk_heatmap.png
Saved: creative_viz_2_profitability.png
Saved: creative_viz_3_segmentation.png


In [15]:
# ============================================================================
# TASK 3: A/B HYPOTHESIS TESTING
# ============================================================================

print("\n" + "="*80)
print("TASK 3: A/B HYPOTHESIS TESTING")
print("="*80)

def prepare_hypothesis_testing_data(df):
    """Prepare data for hypothesis testing"""
    df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)
    df['ClaimFrequency'] = df.groupby('PostalCode')['HasClaim'].transform('mean')
    df['ClaimSeverity'] = df[df['TotalClaims'] > 0].groupby('PostalCode')['TotalClaims'].transform('mean')
    df['Margin'] = df['TotalPremium'] - df['TotalClaims']
    
    return df
df = prepare_hypothesis_testing_data(df)


TASK 3: A/B HYPOTHESIS TESTING


In [16]:
print("\n[3.1] Hypothesis Test 1: Risk Differences Across Provinces")
print("-" * 80)


[3.1] Hypothesis Test 1: Risk Differences Across Provinces
--------------------------------------------------------------------------------


In [17]:
def test_province_risk(df):
    """
    H0: There are no risk differences across provinces
    Risk = Claim Frequency
    """
    print("Testing: H0 - No risk differences across provinces")
    print("Method: Chi-squared test for claim frequency")
    
    # Create contingency table
    contingency_table = pd.crosstab(df['Province'], df['HasClaim'])
    
    # Chi-squared test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\nChi-squared statistic: {chi2:.4f}")
    print(f"P-value: {p_value:.6f}")
    print(f"Degrees of freedom: {dof}")
    
    alpha = 0.05
    if p_value < alpha:
        print(f"\n✓ REJECT H0 (p-value {p_value:.6f} < {alpha})")
        print("Conclusion: There ARE significant risk differences across provinces")
    else:
        print(f"\n✗ FAIL TO REJECT H0 (p-value {p_value:.6f} >= {alpha})")
        print("Conclusion: No significant risk differences across provinces")
    
    # Show claim frequencies by province
    print("\nClaim Frequency by Province:")
    province_risk = df.groupby('Province').agg({
        'HasClaim': 'mean',
        'PolicyID': 'count'
    }).round(4)
    province_risk.columns = ['Claim_Frequency', 'Policy_Count']
    print(province_risk.sort_values('Claim_Frequency', ascending=False))
    
    return chi2, p_value
chi2_province, p_value_province = test_province_risk(df)

Testing: H0 - No risk differences across provinces
Method: Chi-squared test for claim frequency

Chi-squared statistic: 104.1909
P-value: 0.000000
Degrees of freedom: 8

✓ REJECT H0 (p-value 0.000000 < 0.05)
Conclusion: There ARE significant risk differences across provinces

Claim Frequency by Province:
               Claim_Frequency  Policy_Count
Province                                    
Gauteng                 0.0034        393865
KwaZulu-Natal           0.0028        169781
Limpopo                 0.0027         24836
North West              0.0024        143287
Mpumalanga              0.0024         52718
Western Cape            0.0022        170796
Eastern Cape            0.0016         30336
Free State              0.0014          8099
Northern Cape           0.0013          6380


In [18]:
print("\n[3.2] Hypothesis Test 2: Risk Differences Between Zip Codes")
print("-" * 80)

def test_zipcode_risk(df, sample_size=20):
    """
    H0: There are no risk differences between zip codes
    Using ANOVA for multiple groups
    """
    print(f"Testing: H0 - No risk differences between zip codes")
    print(f"Method: ANOVA (using top {sample_size} zip codes by policy count)")
    
    # Get top zip codes by policy count
    top_zipcodes = df['PostalCode'].value_counts().head(sample_size).index
    df_sample = df[df['PostalCode'].isin(top_zipcodes)]
    
    # Prepare groups
    groups = [df_sample[df_sample['PostalCode'] == zc]['HasClaim'].values 
              for zc in top_zipcodes]
    
    # ANOVA test
    f_stat, p_value = f_oneway(*groups)
    
    print(f"\nF-statistic: {f_stat:.4f}")
    print(f"P-value: {p_value:.6f}")
    
    alpha = 0.05
    if p_value < alpha:
        print(f"\n✓ REJECT H0 (p-value {p_value:.6f} < {alpha})")
        print("Conclusion: There ARE significant risk differences between zip codes")
    else:
        print(f"\n✗ FAIL TO REJECT H0 (p-value {p_value:.6f} >= {alpha})")
        print("Conclusion: No significant risk differences between zip codes")
    
    # Show top and bottom zip codes by risk
    print("\nTop 10 Highest Risk Zip Codes:")
    zipcode_risk = df.groupby('PostalCode').agg({
        'HasClaim': 'mean',
        'PolicyID': 'count'
    })
    zipcode_risk = zipcode_risk[zipcode_risk['PolicyID'] >= 10]  # Min 10 policies
    zipcode_risk.columns = ['Claim_Frequency', 'Policy_Count']
    print(zipcode_risk.sort_values('Claim_Frequency', ascending=False).head(10))
    
    return f_stat, p_value
f_stat_zip, p_value_zip = test_zipcode_risk(df)



[3.2] Hypothesis Test 2: Risk Differences Between Zip Codes
--------------------------------------------------------------------------------
Testing: H0 - No risk differences between zip codes
Method: ANOVA (using top 20 zip codes by policy count)

F-statistic: 5.6436
P-value: 0.000000

✓ REJECT H0 (p-value 0.000000 < 0.05)
Conclusion: There ARE significant risk differences between zip codes

Top 10 Highest Risk Zip Codes:
            Claim_Frequency  Policy_Count
PostalCode                               
466                0.055556            18
2920               0.054545            55
1126               0.028571            70
1751               0.025974            77
181                0.025316           158
721                0.025000           120
4027               0.024793           121
1665               0.024390            82
1947               0.022472            89
31                 0.022222           135


In [19]:
print("\n[3.3] Hypothesis Test 3: Margin Differences Between Zip Codes")
print("-" * 80)

def test_zipcode_margin(df, sample_size=20):
    """
    H0: There is no significant margin difference between zip codes
    Using ANOVA
    """
    print(f"Testing: H0 - No significant margin difference between zip codes")
    print(f"Method: ANOVA (using top {sample_size} zip codes)")
    
    # Get top zip codes
    top_zipcodes = df['PostalCode'].value_counts().head(sample_size).index
    df_sample = df[df['PostalCode'].isin(top_zipcodes)]
    
    # Prepare groups
    groups = [df_sample[df_sample['PostalCode'] == zc]['Margin'].values 
              for zc in top_zipcodes]
    
    # ANOVA test
    f_stat, p_value = f_oneway(*groups)
    
    print(f"\nF-statistic: {f_stat:.4f}")
    print(f"P-value: {p_value:.6f}")
    
    alpha = 0.05
    if p_value < alpha:
        print(f"\n✓ REJECT H0 (p-value {p_value:.6f} < {alpha})")
        print("Conclusion: There ARE significant margin differences between zip codes")
    else:
        print(f"\n✗ FAIL TO REJECT H0 (p-value {p_value:.6f} >= {alpha})")
        print("Conclusion: No significant margin differences between zip codes")
    
    # Show margin analysis
    print("\nTop 10 Most Profitable Zip Codes:")
    zipcode_margin = df.groupby('PostalCode').agg({
        'Margin': 'mean',
        'PolicyID': 'count'
    })
    zipcode_margin = zipcode_margin[zipcode_margin['PolicyID'] >= 10]
    zipcode_margin.columns = ['Avg_Margin', 'Policy_Count']
    print(zipcode_margin.sort_values('Avg_Margin', ascending=False).head(10))
    
    return f_stat, p_value

f_stat_margin, p_value_margin = test_zipcode_margin(df)



[3.3] Hypothesis Test 3: Margin Differences Between Zip Codes
--------------------------------------------------------------------------------
Testing: H0 - No significant margin difference between zip codes
Method: ANOVA (using top 20 zip codes)

F-statistic: 1.8775
P-value: 0.011581

✓ REJECT H0 (p-value 0.011581 < 0.05)
Conclusion: There ARE significant margin differences between zip codes

Top 10 Most Profitable Zip Codes:
            Avg_Margin  Policy_Count
PostalCode                          
3887        196.635975            25
4016        195.716263            10
9744        175.104079            50
3802        172.142169            32
1423        171.010857           120
4319        170.421530            54
1932        166.555767            32
2021        165.244211            22
2055        164.889002            16
4140        163.833640            70


In [20]:
print("\n[3.4] Hypothesis Test 4: Risk Differences Between Women and Men")
print("-" * 80)

def test_gender_risk(df):
    """
    H0: There is no significant risk difference between women and men
    Using two-sample t-test
    """
    print("Testing: H0 - No significant risk difference between women and men")
    print("Method: Two-sample t-test and Chi-squared test")
    
    # Filter data
    df_gender = df[df['Gender'].isin(['Male', 'Female'])]
    
    # Chi-squared test for claim frequency
    contingency_table = pd.crosstab(df_gender['Gender'], df_gender['HasClaim'])
    chi2, p_value_chi2, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\nChi-squared Test (Claim Frequency):")
    print(f"Chi-squared statistic: {chi2:.4f}")
    print(f"P-value: {p_value_chi2:.6f}")
    
    # T-test for claim severity (among those who claimed)
    df_claimed = df_gender[df_gender['TotalClaims'] > 0]
    male_claims = df_claimed[df_claimed['Gender'] == 'Male']['TotalClaims']
    female_claims = df_claimed[df_claimed['Gender'] == 'Female']['TotalClaims']
    
    t_stat, p_value_ttest = ttest_ind(male_claims, female_claims)
    
    print(f"\nT-test (Claim Severity):")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value_ttest:.6f}")
    
    alpha = 0.05
    
    # Interpretation
    if p_value_chi2 < alpha or p_value_ttest < alpha:
        print(f"\n✓ REJECT H0")
        print("Conclusion: There ARE significant risk differences between genders")
    else:
        print(f"\n✗ FAIL TO REJECT H0")
        print("Conclusion: No significant risk differences between genders")
    
    # Show statistics by gender
    print("\nRisk Statistics by Gender:")
    gender_stats = df_gender.groupby('Gender').agg({
        'HasClaim': 'mean',
        'TotalClaims': 'mean',
        'TotalPremium': 'mean',
        'PolicyID': 'count'
    }).round(4)
    gender_stats.columns = ['Claim_Frequency', 'Avg_Claim', 'Avg_Premium', 'Policy_Count']
    print(gender_stats)
    
    return chi2, p_value_chi2, t_stat, p_value_ttest

chi2_gender, p_chi2_gender, t_gender, p_t_gender = test_gender_risk(df)


[3.4] Hypothesis Test 4: Risk Differences Between Women and Men
--------------------------------------------------------------------------------
Testing: H0 - No significant risk difference between women and men
Method: Two-sample t-test and Chi-squared test

Chi-squared Test (Claim Frequency):
Chi-squared statistic: 0.0037
P-value: 0.951464

T-test (Claim Severity):
T-statistic: -0.4191
P-value: 0.676016

✗ FAIL TO REJECT H0
Conclusion: No significant risk differences between genders

Risk Statistics by Gender:
        Claim_Frequency  Avg_Claim  Avg_Premium  Policy_Count
Gender                                                       
Female           0.0021    37.0461      45.0748          6755
Male             0.0022    32.6203      36.9046         42817


In [21]:
print("\n[3.5] Summary of Hypothesis Tests")
print("-" * 80)

def summarize_hypothesis_tests(results):
    """Summarize all hypothesis test results"""
    summary = pd.DataFrame({
        'Hypothesis': [
            'H0: No risk differences across provinces',
            'H0: No risk differences between zip codes',
            'H0: No margin differences between zip codes',
            'H0: No risk differences between genders'
        ],
        'Test_Type': ['Chi-squared', 'ANOVA', 'ANOVA', 'Chi-squared & T-test'],
        'P_Value': results,
        'Decision': ['Reject' if p < 0.05 else 'Fail to Reject' for p in results],
        'Significance': ['Yes' if p < 0.05 else 'No' for p in results]
    })
    
    print("\nHypothesis Testing Summary:")
    print(summary.to_string(index=False))
    
    return summary

p_values = [p_value_province, p_value_zip, p_value_margin, p_chi2_gender]
summary_df = summarize_hypothesis_tests(p_values)



[3.5] Summary of Hypothesis Tests
--------------------------------------------------------------------------------

Hypothesis Testing Summary:
                                 Hypothesis            Test_Type      P_Value       Decision Significance
   H0: No risk differences across provinces          Chi-squared 5.925511e-19         Reject          Yes
  H0: No risk differences between zip codes                ANOVA 2.590639e-14         Reject          Yes
H0: No margin differences between zip codes                ANOVA 1.158103e-02         Reject          Yes
    H0: No risk differences between genders Chi-squared & T-test 9.514645e-01 Fail to Reject           No


In [22]:
# ============================================================================
# TASK 4: MACHINE LEARNING & STATISTICAL MODELING
# ============================================================================

print("\n" + "="*80)
print("TASK 4: MACHINE LEARNING & STATISTICAL MODELING")
print("="*80)

print("\n[4.1] Data Preparation for Modeling")
print("-" * 80)

def prepare_modeling_data(df):
    """Prepare data for machine learning models"""
    
    print("Preparing data for modeling...")
    
    # Create a copy
    df_model = df.copy()
    
    # Feature engineering
    df_model['VehicleAge'] = 2015 - df_model['RegistrationYear']
    df_model['HasClaim'] = (df_model['TotalClaims'] > 0).astype(int)
    df_model['LogPremium'] = np.log1p(df_model['TotalPremium'])
    df_model['LogClaims'] = np.log1p(df_model['TotalClaims'])
    
    # Handle missing values
    numerical_cols = df_model.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        df_model[col].fillna(df_model[col].median(), inplace=True)
    
    categorical_cols = df_model.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_model[col].fillna(df_model[col].mode()[0], inplace=True)
    
    print(f"Data prepared: {df_model.shape}")
    print(f"Features: {df_model.shape[1]}")
    
    return df_model
df_model = prepare_modeling_data(df)


TASK 4: MACHINE LEARNING & STATISTICAL MODELING

[4.1] Data Preparation for Modeling
--------------------------------------------------------------------------------
Preparing data for modeling...
Data prepared: (1000098, 59)
Features: 59


In [23]:
print("\n[4.2] Feature Selection and Encoding")
print("-" * 80)

def encode_features(df):
    """Encode categorical features"""
    
    df_encoded = df.copy()
    
    # Select features for modeling
    categorical_features = [
        'Province', 'VehicleType', 'Make', 'Gender', 
        'MaritalStatus', 'CoverType', 'AlarmImmobiliser'
    ]
    
    numerical_features = [
        'VehicleAge', 'SumInsured', 'CalculatedPremiumPerTerm',
        'Cylinders', 'Kilowatts', 'NumberOfDoors', 
        'CustomValueEstimate', 'CapitalOutstanding'
    ]
    
    # Label encoding for categorical variables
    label_encoders = {}
    for col in categorical_features:
        if col in df_encoded.columns:
            le = LabelEncoder()
            df_encoded[col + '_encoded'] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le
    
    # Create feature list
    feature_cols = [col + '_encoded' for col in categorical_features if col in df_encoded.columns]
    feature_cols += [col for col in numerical_features if col in df_encoded.columns]
    
    print(f"Total features for modeling: {len(feature_cols)}")
    print(f"Categorical features encoded: {len(categorical_features)}")
    print(f"Numerical features: {len(numerical_features)}")
    
    return df_encoded, feature_cols, label_encoders

df_encoded, feature_cols, label_encoders = encode_features(df_model)



[4.2] Feature Selection and Encoding
--------------------------------------------------------------------------------
Total features for modeling: 13
Categorical features encoded: 7
Numerical features: 8


In [ ]:
print("\n[4.3] Model 1: Linear Regression for Total Claims (by Zip Code)")
print("-" * 80)

def linear_regression_by_zipcode(df, feature_cols):
    """
    Fit linear regression model for each zipcode to predict total claims
    """
    
    print("Building Linear Regression models by Zip Code...")
    
    # Get top zip codes by policy count
    top_zipcodes = df['PostalCode'].value_counts().head(10).index
    
    results = []
    
    for zipcode in top_zipcodes:
        df_zip = df[df['PostalCode'] == zipcode].copy()
        
        if len(df_zip) < 30:  # Skip if too few samples
            continue
        
        # Prepare features and target
        X = df_zip[feature_cols].fillna(0)
        y = df_zip['TotalClaims']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Train model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        
        # Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        results.append({
            'ZipCode': zipcode,
            'Sample_Size': len(df_zip),
            'RMSE': rmse,
            'R2_Score': r2,
            'MAE': mae
        })
        
        print(f"ZipCode {zipcode}: RMSE={rmse:.2f}, R²={r2:.4f}, MAE={mae:.2f}")
    
    results_df = pd.DataFrame(results)
    print("\nSummary of Linear Regression Models:")
    print(results_df.describe())
    
    return results_df

lr_results = linear_regression_by_zipcode(df_encoded, feature_cols)